In [27]:
# ----------------------------
# (A) Problem setup
# ----------------------------
edges = [
    ("A","B",2), ("A","C",3),
    ("B","D",5), ("B","E",2),
    ("C","F",1),
    ("D","G",2),
    ("E","G",3),
    ("F","G",1),
]

h = {"A":4, "B":2, "C":1.5, "D":0.5, "E":1, "F":1, "G":0}
start, goal = "A", "G"

# adjacency list
adj = {}
for u,v,c in edges:
    adj.setdefault(u, []).append((v,c))

# ----------------------------
# (B) IDA* pseudocode (your version) - line by line
# ----------------------------
code_lines = [
    "path            current search path (acts like a stack)",
    "n            current node (last node in current path)",
    "g               the cost to reach current node",
    "f               estimated cost of the cheapest path (root..node..goal)",
    "h(n)         estimated cost of the cheapest path (node..goal)",
    "cost(n, v) step cost function",
    "is_goal(n)   goal test",
    "successors(n) node expanding function, expand nodes ordered by g + h(n)",
    "ida_star(root)  return either NOT_FOUND or a pair with the best path and its cost",
    "",
    "procedure ida_star(root)",
    "    cutoff := h(root)",
    "    path  := [root]",
    "    loop",
    "        t := search(path, 0, cutoff)",
    "        if t = GOAL then return (path, cutoff)",
    "        if t = ∞ then return NOT_FOUND",
    "        cutoff := t",
    "",
    "function search(path, g, cutoff)",
    "    n := path.last",
    "    f := g + h(n)",
    "    if f > cutoff: return f",
    "    if is_goal(n): return GOAL",
    "    min := ∞",
    "    for v in successors(n) do",
    "        if v not in path: ",
    "            path.push(v)",
    "            t := search(path, g + cost(n, v), cutoff)",
    "            if t = GOAL: return GOAL",
    "            if t < min: min := t",
    "            path.pop()",
    "    return min",
]

# line index helper (so we can highlight by name)
L = {s:i for i,s in enumerate(code_lines)}  # may collide if duplicate; we use manual indices below.


In [28]:
import math
from dataclasses import dataclass
from typing import Any, Dict, List, Tuple, Optional

GOAL = "GOAL"

def LI(s: str) -> int:
    """Line index (0-based) for exact pseudocode line."""
    return code_lines.index(s)


# --- choose successors ordered by f(child)= (g+cost) + h(child) ---
def successors_ordered(n: str, g: float) -> List[Tuple[str, float]]:
    succs = adj.get(n, [])
    # sort by (g+cost+h(v)) ascending, tie by name
    return sorted(succs, key=lambda vc: (g + vc[1] + h[vc[0]], vc[0]))

# --- pretty infinity ---
INF = float("inf")

# ----------------------------
# Trace record (one "step" for animation)
# ----------------------------
@dataclass
class Step:
    line_idx: int
    note: str
    row: Dict[str, Any]  # one row of the "terminal table"

def make_depth_cols(call_stack: List[int], max_depth_cols: int = 4) -> Dict[str, Any]:
    """
    Turn current call stack [id0,id1,id2,...] into columns:
    call@d0, call@d1, ...
    """
    out = {}
    for d in range(max_depth_cols):
        out[f"call@d{d}"] = call_stack[d] if d < len(call_stack) else "/"
    return out

steps: List[Step] = []

# global call id generator
call_id = 0

def log_step(line_idx: int,
             call_stack: List[int],
             path: List[str],
             n: Optional[str],
             succs: Optional[List[str]],
             gval: Any,
             cutoff: float,
             min_b: Any,
             tval: Any,
             note: str,
             v: Optional[str]="",              # ✅ NEW
             cost_nv: Any = ""              # ✅ optional
            ):
    row = {}
    row.update(make_depth_cols(call_stack, max_depth_cols=6))
    row.update({
        "path": str(path),
        "n": n if n is not None else "",
        "v": v if v is not None else "",                 # ✅ NEW
        "cost(n,v)": cost_nv,                             # ✅ optional
        "successors(n)": str(succs) if succs is not None else "",
        "g": gval if gval is not None else "",
        "h(n)": h[n] if (n is not None and n in h and isinstance(gval,(int,float))) else (h[n] if (n is not None and n in h) else ""),
        "f=g+h": (gval + h[n]) if (n is not None and isinstance(gval,(int,float))) else "",
        "cutoff": cutoff,
        "min": min_b,
        "t": tval,
    })
    steps.append(Step(line_idx=line_idx, note=note, row=row))



# ----------------------------
# IDA* with tracing (very close to your pseudocode)
# ----------------------------
def ida_star_with_trace(root: str, goal: str):
    global call_id
    call_id = 0

    cutoff = h[root]
    path = [root]

    # cutoff := h(root)  
    log_step(
        line_idx=LI("    cutoff := h(root)"),
        call_stack=[],
        path=path,
        n=None,               # ✅ no n yet
        succs=None,
        gval=None,            # ✅ no g yet
        cutoff=cutoff,
        min_b="",
        tval="",
        note=f"Initialize cutoff = h({root}) = {cutoff}"
    )

    while True:
        # t := search(path, 0, cutoff)
        log_step(
            line_idx=LI("        t := search(path, 0, cutoff)"),
            call_stack=[],
            path=path,
            n=None,            # ✅ still no n (search hasn't started)
            succs=None,
            gval=None,
            cutoff=cutoff,
            min_b="",
            tval="",
            note=f"Call search(path, g=0, cutoff={cutoff})"
        )

        t, found = search_with_trace(path, g=0, cutoff=cutoff, goal=goal, call_stack=[])

        # if t = GOAL then return (path, cutoff)
        if found:
            log_step(
                line_idx=LI("        if t = GOAL then return (path, cutoff)"),
                call_stack=[],
                path=path,
                n=None,
                succs=None,
                gval=None,
                cutoff=cutoff,
                min_b="",
                tval="GOAL",

                note=f"t=GOAL -> return solution path={path} with cost={cutoff}"
            )
            return path, cutoff

        # if t = ∞ then return NOT_FOUND
        if t == INF:
            log_step(
                line_idx=LI("        if t = ∞ then return NOT_FOUND"),
                call_stack=[],
                path=path,
                n=None,
                succs=None,
                gval=None,
                cutoff=cutoff,
                min_b="",
                tval=INF,
                min_after="",
                note="t=∞ -> NOT_FOUND"
            )
            return None, None

        # cutoff := t
        log_step(
            line_idx=LI("        cutoff := t"),
            call_stack=[],
            path=path,
            n=None,
            succs=None,
            gval=None,
            cutoff=cutoff,
            min_b="",
            tval=t,
            note=f"Increase cutoff: {cutoff} -> {t}"
        )
        cutoff = t

def search_with_trace(path: List[str], g: float, cutoff: float, goal: str, call_stack: List[int]):
    global call_id
    call_id += 1
    my_id = call_id
    call_stack = call_stack + [my_id]

    # n := path.last
    n = path[-1]
    log_step(
        line_idx=LI("    n := path.last"),
        call_stack=call_stack, path=path, n=n, succs=None,
        gval="", cutoff=cutoff, min_b="", tval="",
        note=f"Enter search: set n = path.last = {n}"
    )

    # f := g + h(n)
    f = g + h[n]
    log_step(
        line_idx=LI("    f := g + h(n)"),
        call_stack=call_stack, path=path, n=n, succs=None,
        gval=g, cutoff=cutoff, min_b="", tval="",
        note=f"Compute f = g + h(n) = {g} + {h[n]} = {f}"
    )

    # if f > cutoff: return f
    if f > cutoff:
        log_step(
            line_idx=LI("    if f > cutoff: return f"),
            call_stack=call_stack, path=path, n=n, succs=None,
            gval=g, cutoff=cutoff, min_b="", tval="",
            note=f"Prune: f={f} > cutoff={cutoff}, return f"
        )
        return f, False

    # if is_goal(n): return GOAL
    if n == goal:
        log_step(
            line_idx=LI("    if is_goal(n): return GOAL"),
            call_stack=call_stack, path=path, n=n, succs=None,
            gval=g, cutoff=cutoff, min_b="", tval="GOAL", 
            note="Goal test true -> return GOAL"
        )
        return 0, True

    # min := ∞
    min_exceed = INF
    log_step(
        line_idx=LI("    min := ∞"),
        call_stack=call_stack, path=path, n=n, succs=None,
        gval=g, cutoff=cutoff, min_b=INF, tval="", 
        note="Initialize min := ∞"
    )

    # for v in successors(n) do
    ordered = successors_ordered(n, g)
    succ_list = [v for v,_ in ordered]
    log_step(
        line_idx=LI("    for v in successors(n) do"),
        call_stack=call_stack, path=path, n=n, succs=succ_list,
        gval=g, cutoff=cutoff, min_b=min_exceed, tval="", 
        note=f"Expand successors(n) ordered by f(child)=g+cost+h: {succ_list}"
    )

    for v, cost_nv in ordered:
        # if v not in path:
        cond = (v not in path)
        log_step(
            line_idx=LI("        if v not in path: "),
            call_stack=call_stack, path=path, n=n, v=v, succs=succ_list,
            gval=g, cutoff=cutoff, min_b=min_exceed, tval="", 
            note=f"Check v not in path? v={v} -> {cond}"
        )
        if not cond:
            continue

        # path.push(v)
        path.append(v)
        log_step(
            line_idx=LI("            path.push(v)"),
            call_stack=call_stack,
            path=path,
            n=n,                      # ✅ still the parent's n
            v=v,                      # ✅ show loop variable v
            succs=succ_list,
            gval=g,                   # ✅ g is still parent's g here
            cutoff=cutoff,
            min_b=min_exceed,
            tval="",
            cost_nv=cost_nv,          # ✅ show edge cost
            note=f"path.push({v}) (still in same search frame; n stays {n})"
        )

        # t := search(path, g + cost(n, v), cutoff)
        g2 = g + cost_nv
        log_step(
            line_idx=LI("            t := search(path, g + cost(n, v), cutoff)"),
            call_stack=call_stack,
            path=path,
            n=n,                      # ✅ still parent's n at the CALL site
            v=v,                      # ✅ calling with this v
            succs=succ_list,
            gval=g,                   # ✅ still parent's g at call site (optional)
            cutoff=cutoff,
            min_b=min_exceed,
            tval="",
            cost_nv=cost_nv,
            note=f"Call search on child v={v}: new g will be {g2}"
        )
        t, found = search_with_trace(path, g=g2, cutoff=cutoff, goal=goal, call_stack=call_stack)

        if found:
            log_step(
                line_idx=LI("            if t = GOAL: return GOAL"),
                call_stack=call_stack, path=path, n=path[-1], succs=None,
                gval=g2, cutoff=cutoff, min_b=min_exceed, tval="GOAL", 
                note="Child returned GOAL -> propagate GOAL upward"
            )
            return 0, True
        else:
            log_step(
                line_idx=LI("            if t = GOAL: return GOAL"),
                call_stack=call_stack, path=path, n=n, succs=succ_list,
                gval=g, cutoff=cutoff, min_b=min_exceed, tval=t, 
                note=""
            )

        before = min_exceed
        if t < min_exceed:
            min_exceed = t
        log_step(
            line_idx=LI("            if t < min: min := t"),
            call_stack=call_stack, path=path, n=n, succs=succ_list,
            gval=g, cutoff=cutoff, min_b=min_exceed, tval=t, 
            note=f"Update min: {before} -> {min_exceed}"
        )

        popped = path.pop()
        log_step(
            line_idx=LI("            path.pop()"),
            call_stack=call_stack, path=path, n=n, succs=succ_list,
            gval=g, cutoff=cutoff, min_b=min_exceed, tval=t, 
            note=f"path.pop() -> removed {popped}"
        )

    log_step(
        line_idx=LI("    return min"),
        call_stack=call_stack, path=path, n=n, succs=succ_list,
        gval=g, cutoff=cutoff, min_b=min_exceed, tval="", 
        note=f"Return min={min_exceed} to caller"
    )
    return min_exceed, False

# run once to build steps + final solution
solution_path, sol_cost = ida_star_with_trace(start, goal)
solution_path, sol_cost, len(steps)


(['A', 'C', 'F', 'G'], 5, 171)

In [29]:
import networkx as nx

G = nx.DiGraph()
for u, v, c in edges:
    G.add_edge(u, v, cost=c)

def get_graphviz_pos(G, root="A"):
    try:
        from networkx.drawing.nx_agraph import graphviz_layout
        H = G.copy()
        H.graph["graph"] = {"rankdir": "TB"}  # top-bottom
        return graphviz_layout(H, prog="dot", root=root)
    except Exception as e1:
        try:
            from networkx.drawing.nx_pydot import graphviz_layout
            H = G.copy()
            H.graph["graph"] = {"rankdir": "TB"}
            return graphviz_layout(H, prog="dot")
        except Exception as e2:
            raise RuntimeError(
                "graphviz_layout unavailable. Please install pygraphviz (recommended) "
                "or pydot+graphviz system package."
            ) from e2

pos = get_graphviz_pos(G, root="A")


In [30]:
import io, base64, ast
import matplotlib.pyplot as plt

def _parse_list_from_str(s):
    """Safely parse something like "['A','B']" produced by str(list)."""
    if not s or s == "": 
        return []
    try:
        val = ast.literal_eval(s)
        return val if isinstance(val, list) else []
    except Exception:
        return []

def draw_tree_step_b64(G, pos, step_row):
    """
    step_row: one row dict from your 'steps' trace.
    Highlights:
      - path nodes + path edges: recursion stack / current path in memory
      - current node: white thick outline
      - successors(n): nodes being considered in current stack frame
    """
    path = _parse_list_from_str(step_row.get("path", ""))
    n = step_row.get("n", None)
    if n in ("", None):
        n = None
    succs = _parse_list_from_str(step_row.get("successors(n)", ""))

    path_nodes = set(path)
    succ_nodes = set(succs) - path_nodes  # successors not already on path
    current = n

    # edges on current path
    path_edges = set()
    for a, b in zip(path, path[1:]):
        if G.has_edge(a, b):
            path_edges.add((a, b))

    fig, ax = plt.subplots(figsize=(6.2, 6))
    ax.axis("off")

    # --- draw base edges (thin) ---
    all_edges = list(G.edges())
    # --- draw base directed edges with visible arrows ---
    nx.draw_networkx_edges(
        G, pos, edgelist=all_edges, ax=ax,
        arrows=True,
        arrowstyle='-|>',          # clear arrow head
        arrowsize=22,              # bigger arrow head
        width=1.4,
        min_source_margin=18,      # keep arrow away from node circle
        min_target_margin=18,
        connectionstyle="arc3,rad=0.0"
    )

    # --- draw highlighted path edges (thicker) ---
    if path_edges:
        nx.draw_networkx_edges(
            G, pos, edgelist=list(path_edges), ax=ax,
            arrows=True,
            arrowstyle='-|>',
            arrowsize=26,
            width=3.8,
            min_source_margin=18,
            min_target_margin=18,
            connectionstyle="arc3,rad=0.0"
        )


    # We'll use distinct fills so it’s visually clear.
    base_nodes = set(G.nodes()) - path_nodes - succ_nodes
    # draw base nodes
    nx.draw_networkx_nodes(G, pos, nodelist=list(base_nodes), ax=ax, node_size=900, linewidths=1.5)
    # draw successor nodes (blue-ish)
    nx.draw_networkx_nodes(G, pos, nodelist=list(succ_nodes), ax=ax, node_size=900, linewidths=1.8, node_color="#9ecbff")
    # draw path nodes (yellow-ish)
    nx.draw_networkx_nodes(G, pos, nodelist=list(path_nodes), ax=ax, node_size=900, linewidths=1.8, node_color="#ffd84d")

    # current node: white border highlight (draw an overlay ring)
    if current in G.nodes():
        nx.draw_networkx_nodes(
            G, pos, nodelist=[current], ax=ax,
            node_size=980, linewidths=3.2, node_color="none", edgecolors="red"
        )

    # labels
    nx.draw_networkx_labels(G, pos, ax=ax, font_size=12, font_weight="bold")

    # edge labels (cost)
    edge_labels = {(u, v): G[u][v]["cost"] for u, v in G.edges()}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax, font_size=11)

    # legend-ish text
    ax.text(
        0.0, 0,
        "Memory highlight: PATH (yellow) + successors(n) (blue) + current n (red border)",
        transform=ax.transAxes, fontsize=10
    )

    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=180, bbox_inches="tight")
    plt.close(fig)
    buf.seek(0)
    return base64.b64encode(buf.read()).decode("ascii")


In [31]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML


def render_code_highlight(lines, hl_idx):
    out = []
    for i, s in enumerate(lines):
        cls = "hl" if i == hl_idx else ""
        out.append(
            f'<div class="codeline {cls}" id="codeL{i}">'
            f'<span class="lineno">{i+1:>3d}│</span>{s}</div>'
        )
    return "\n".join(out)


CSS = """
<style>
/* overall layout */
.wrap {
  display: flex;
  flex-direction: column;
  gap: 14px;
}

/* top row: code (50%) | info (50%) */
.toprow {
  display: flex;
  gap: 14px;
  align-items: flex-start;
}

.panel {
  border: 1px solid #222;
  border-radius: 12px;
  padding: 12px;
}

/* pseudocode panel: 50% */
.codepanel {
  background: #0b0b0b;
  color: #e8e8e8;
  width: 50%;
  max-height: 400px;
  overflow: auto;
  font-family: ui-monospace, Menlo, Consolas, monospace;
  font-size: 14px;
  line-height: 1.35;
}

/* info panel: 50% total, split inside */
.infopanel {
  background: #ffffff;
  color: #000000;
  width: 50%;
  max-height: 500px;
  overflow: hidden;
  padding: 0;           /* we'll pad inside columns */
}

/* inside infopanel: graph (50% of infopanel) | htable (50% of infopanel) */
.infogrid {
  display: flex;
  gap: 12px;
  height: 100%;
  padding: 12px;
  box-sizing: border-box;
}

.graphbox, .htablebox {
  width: 50%;
  overflow: auto;
}

.graphimg {
  width: 100%;
  height: auto;
  display: block;
}

/* bottom: trace table full width */
.tablepanel {
  background: #ffffff;
  color: #000000;
  width: 100%;
  max-height: 340px;
  overflow: auto;
}

/* code line styles */
.codeline {
  white-space: pre;
  padding: 2px 6px;
  border-radius: 6px;
}

.hl {
  background: #ffd84d;
  color: #111;
  font-weight: 700;
}

.lineno {
  display: inline-block;
  width: 3.2em;
  color: #666;
  user-select: none;
}

.note {
  margin-top: 8px;
  color: #ddd;
  font-family: ui-sans-serif, system-ui;
}
</style>
"""


In [ ]:


# Build cumulative rows for display
rows = [st.row for st in steps]

# Controls
slider = widgets.IntSlider(value=0, min=0, max=len(steps)-1, step=1, description="Step", continuous_update=False)
play = widgets.Play(value=0, min=0, max=len(steps)-1, step=1, interval=700)
widgets.jslink((play, "value"), (slider, "value"))
btn_prev = widgets.Button(description="◀ Prev", layout=widgets.Layout(width="90px"))
btn_next = widgets.Button(description="Next ▶", layout=widgets.Layout(width="90px"))

out = widgets.Output()

def show(step_idx):
    step_idx = max(0, min(step_idx, len(steps)-1))
    st = steps[step_idx]

    K = 12
    start_i = max(0, step_idx - K + 1)
    df = pd.DataFrame(rows[start_i:step_idx+1]).iloc[::-1]

    # pseudocode highlight
    code_html = render_code_highlight(code_lines, st.line_idx)

    # dynamic graph b64 for this step
    tree_b64 = draw_tree_step_b64(G, pos, st.row)

    # h table (static)
    h_df = pd.DataFrame({
        "node": list(h.keys()),
        "h(n)": list(h.values())
    }).sort_values("node")

    h_html = f"""
    <div style="font-weight:700; margin-bottom:6px;">Heuristic table h(n)</div>
    {h_df.to_html(index=False)}
    """


    graph_html = f"""
    <div style="font-weight:700; margin-bottom:6px;">Rooted tree (top-down, root=A)</div>
    <img class="graphimg" src="data:image/png;base64,{tree_b64}" />
    """

    # info panel: graph | h-table 
    info_html = f"""
    <div class="infogrid">
      <div class="graphbox">{graph_html}</div>
      <div class="htablebox">{h_html}</div>
    </div>
    """

    scroll_js = f"""
    <script>
    (function() {{
      const el = document.getElementById("codeL{st.line_idx}");
      if (el) el.scrollIntoView({{block:"center", inline:"nearest"}});
    }})();
    </script>
    """


    with out:
        out.clear_output(wait=True)
        display(HTML(CSS + f"""
        <div class="wrap">

          <div class="toprow">
            <div class="panel codepanel">{code_html}</div>
            <div class="panel infopanel">{info_html}</div>
          </div>

          <div class="panel tablepanel">{df.to_html(index=False)}</div>

        </div>

        <div class="note"><b>Note:</b> {st.note}</div>
        <div class="note"><b>Solution:</b> {solution_path}  &nbsp;&nbsp; <b>Cost:</b> {sol_cost}</div>
        """ + scroll_js))


def on_slider(change):
    if change["name"] == "value":
        show(change["new"])

def on_prev(_):
    slider.value = max(0, slider.value - 1)

def on_next(_):
    slider.value = min(len(steps)-1, slider.value + 1)

slider.observe(on_slider)
btn_prev.on_click(on_prev)
btn_next.on_click(on_next)

display(widgets.HBox([play, slider, btn_prev, btn_next]), out)
show(0)


Output()